In [1]:
# Dependencies
import pandas as pd
from pathlib import Path

from config import driver, username, password, host, port, database
from sqlalchemy import create_engine

In [2]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [ ]:
QuestionsListDF = pd.read_sql_table('questionslist', connection)
QuestionsListDF.head()

In [3]:
# pd.options.display.max_columns = 300
pd.options.display.max_columns = 40

In [4]:
# local csv file read

QuestionnaireDF = pd.read_csv(Path('../resources/data.csv'), delimiter='\t')
QuestionnaireDF.rename(columns ={'IE':'introvert_extrovert'}, inplace=True)
QuestionnaireDF.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,Q4I,Q4E,Q5A,Q5I,Q5E,Q6A,Q6I,Q6E,Q7A,Q7I,...,Q88I,Q88E,Q89A,Q89I,Q89E,Q90A,Q90I,Q90E,Q91A,Q91I,Q91E,country,dateload,introelapse,testelapse,surveyelapse,gender,engnat,age,introvert_extrovert
0,5,51,7107,3,91,2522,1,56,6180,2,66,5365,3,74,6646,2,23,2726,3,11,...,67,883,4,4,11548,3,40,4648,3,35,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,5,39,6354,5,13,3092,1,12,5243,5,3,8563,2,75,4027,2,79,2885,5,54,...,80,5095,4,90,2827,4,28,3884,3,1,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,17,5397,4,35,2747,5,40,5262,3,80,3822,4,21,4338,5,68,2359,5,42,...,46,2935,2,41,3337,1,87,1759,1,19,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,5,41,3055,2,14,3348,1,13,5141,1,42,2925,5,33,2478,5,8,1757,5,63,...,2,5562,4,40,3198,3,15,2345,3,23,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,1,76,2542,2,54,1878,1,15,5637,1,13,2020,3,59,3082,3,44,3024,5,49,...,33,7224,5,7,4024,5,86,6413,5,69,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2


In [ ]:
# Read from AWS

# QuestionnaireDF = pd.read_sql_table('questionnaire', connection)
# QuestionnaireDF.head()

## Steps to be performed

QuestionnaireDF.dtypesPreprocessing - 
1) Find outlier who took more time to respond to a question - Sushma
2) Age binning - Henry
3) Identifying the millenials age groups - Scott
3) Age cleanup - Henry
4) Inspect Countries --> Has NONE value  -> Consider them as Other --> Eliminate Countries from training - Scott

Question:
1) Introvert = 1, Extrovert = 2, No = 3 ---> Use Introvert and Extrovert data to train and test the model?


Machine learning - 
1) RandomForestClassifier --> feature importance
2) Logistic Regression

Visualizations - 

1) top questions where people are consistently either 1 or 5 about their responses
2) Introvert Vs Extrovert Vs NotToIdentify - Male Vs Female
3) Demographics ( Example - Age, Gender, Country %age, english speaking vs No)
3a) Age-wise question --> Introvert Vs Extrovert by millenials (demographics) 
4) Boxplot - of lapsed time

In [5]:
# HT
QuestionnaireDF.dtypes

Q1A                    int64
Q1I                    int64
Q1E                    int64
Q2A                    int64
Q2I                    int64
                       ...  
surveyelapse           int64
gender                 int64
engnat                 int64
age                    int64
introvert_extrovert    int64
Length: 282, dtype: object

In [6]:
# HT
QuestionnaireDF.shape

(7188, 282)

In [ ]:
# HT
# Age: Clean up invalid age range above max_age
max_age = 100
age_range = (QuestionnaireDF['age'] < max_age)
QuestionnaireDF = QuestionnaireDF.loc[age_range]

In [24]:

y = [x for x in range(10,101,10)]
y

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [26]:
# HT
# Age: Categorize ages into bins
# bincount = 8
bindefinition = [x for x in range(10, 101, 10)]
q2 = pd.cut(QuestionnaireDF['age'], bins=bindefinition)
q2

0       (20, 30]
1       (20, 30]
2       (10, 20]
3       (20, 30]
4       (10, 20]
          ...   
7183    (50, 60]
7184    (10, 20]
7185    (20, 30]
7186    (10, 20]
7187    (20, 30]
Name: age, Length: 7188, dtype: category
Categories (9, interval[int64]): [(10, 20] < (20, 30] < (30, 40] < (40, 50] ... (60, 70] < (70, 80] < (80, 90] < (90, 100]]

In [30]:
pd.get_dummies(q2)

,"(10, 20]","(20, 30]","(30, 40]","(40, 50]","(50, 60]","(60, 70]","(70, 80]","(80, 90]","(90, 100]"
0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
7183,0,0,0,0,1,0,0,0,0
7184,1,0,0,0,0,0,0,0,0
7185,0,1,0,0,0,0,0,0,0
7186,1,0,0,0,0,0,0,0,0


In [ ]:
QuestionnaireDF.shape

## Preprocessing - Provide output as CleansedDF

In [ ]:
CleansedDF = QuestionnaireDF.drop(columns=['country', 'dateload', 'introelapse', 'testelapse', 'surveyelapse','introvert_extrovert'])
CleansedDF.head()

In [ ]:
QuestionnaireDF['age'].sort_values().unique()

In [ ]:
QuestionnaireDF.nunique()

## Start Machine learning - Runs based on DF named CleansedDF

In [ ]:
ValidationDF = CleansedDF.loc[QuestionnaireDF['introvert_extrovert'] == 3]
ValidationDF.head()

In [ ]:
TrainTestDataDF = CleansedDF.loc[(QuestionnaireDF['introvert_extrovert'] == 1) | (QuestionnaireDF['introvert_extrovert'] == 2)]
TrainTestDataDF

In [ ]:
y = CleansedDF['introvert_extrovert']
y